In [1]:
import ray
import pickle
import sys
import numpy as np
import math
import os
#my module
from LDPC_code import LDPC_construction
from LDPC_code import LDPC_encode
from LDPC_code import LDPC_decode
from polar_code import polar_construction
from polar_code import polar_encode
from polar_code import polar_decode
from polar_code import RCA
from polar_code import iGA
from polar_code import monte_carlo_construction
from turbo_code import turbo_construction
from turbo_code import turbo_encode
from turbo_code import turbo_decode
from modulation import modulation
from modulation.BICM import make_BICM
from channel import AWGN

In [40]:
class Mysystem_LDPC():
    def __init__(self,M,K):
        self.M=M
        self.K=K
        #self.N=self.K*int(np.log2(self.M))
        self.N=self.K*2
        self.BICM=True 
        self.BICM_ID=True
        
        if self.BICM_ID==True:
            self.BICM=True
            self.BICM_ID_itr=10
                
        #coding
        self.cd=LDPC_construction.coding(self.N,self.K)
        self.ec=LDPC_encode.encoding(self.cd)
        self.dc=LDPC_decode.decoding(self.cd)
        #modulation
        self.modem=modulation.QAMModem(self.M)
        

        #channel
        self.ch=AWGN._AWGN()
        
        #filename
        self.filename="LDPC_code_{}_{}_{}".format(self.N,self.K,self.M)
        if self.BICM==True:
            self.BICM_int,self.BICM_deint=make_BICM(self.N,self.M)
            self.filename=self.filename+"_BICM"
        
        #output filename to confirm which program I run
        print(self.filename)
    
    def main_func(self,EsNodB):
        EsNo = 10 ** (EsNodB / 10)
        No=1/EsNo

        info,cwd=self.ec.LDPC_encode()
        info=cwd #BICMのとき、cwdがインターリーブされてしまい、比較できなくなる為、infoをcwdに変更する
        if self.BICM==True:
            cwd=cwd[self.BICM_int]
        TX_conste=self.modem.modulate(cwd)
        
        #channel
        RX_conste=self.ch.add_AWGN(TX_conste,No)
        
        #at the reciever
        if self.BICM_ID==False:
            Lc=self.modem.demodulate(RX_conste,No)
            if self.BICM==True:
                Lc=Lc[self.BICM_deint]
            EST_cwd,_=self.dc.LDPC_decode(Lc)
            return info,EST_cwd
        
        
        elif self.BICM_ID==True:
            
            #demodulate      
            Lc,[zeros,ones]=self.modem.demodulate(RX_conste,No,self.BICM_ID)
            
            ###check
            num=self.modem.calc_exp(zeros,No)
            denum=self.modem.calc_exp(ones,No)
            Lc_check=(np.transpose(num[0]) - np.transpose(denum[0])).ravel(order='F')
            #print(Lc3)
            if np.any(Lc!=Lc_check):
                print("Lc is different")
            ###check end
            
            return Lc,[zeros,ones]

In [3]:
def key_preparation(self):
    """ Creates the coordinates
    where either zeros or ones can be placed in the signal constellation..
    Returns
    -------
    zeros : list of lists of complex values
        The coordinates where zeros can be placed in the signal constellation.
    ones : list of lists of complex values
        The coordinates where ones can be placed in the signal constellation.
    """

    zeros = [[] for i in range(self.N)]
    ones = [[] for i in range(self.N)]

    bin_seq = self.de2bin(self.m)

    for bin_idx, bin_symb in enumerate(bin_seq):
        if self.bin_input == True:
            key = bin_symb
        else:
            key = bin_idx
        for possition, digit in enumerate(bin_symb):
            if digit == '0':
                zeros[possition].append(key)
            else:
                ones[possition].append(key)
    
    #from str list to int array 
    for i in range(len(zeros)):
        zeros[i]=np.array([int(s, 2) for s in zeros[i]])
        ones[i]=np.array([int(s, 2) for s in ones[i]])
                
    return zeros, ones

In [4]:
K=512
M=4
EsNodB=0
EsNo = 10 ** (EsNodB / 10)
No=1/EsNo
mysys=Mysystem_LDPC(M,K)
modem=mysys.modem
modem.code_book

(2, 4)
LDPC_code_1024_512_4_BICM


{'00': (-0.7071067811865475+0.7071067811865475j),
 '01': (-0.7071067811865475-0.7071067811865475j),
 '11': (0.7071067811865475-0.7071067811865475j),
 '10': (0.7071067811865475+0.7071067811865475j)}

In [5]:
np.arange(1024)

array([   0,    1,    2, ..., 1021, 1022, 1023])

In [12]:
def create_mat():
    '''
    return 2D numpy array
    in which the first column is 0(bin)
    second column is1(bin),,,, 
    '''
    #print(str(0)+str(modem.N)+'b')
    #print(format(4,str(0)+str(modem.N)+'b'))
    mat=np.zeros((modem.M,modem.N),dtype=int)
    for i in range(modem.M):
        digit_num=str(0)+str(modem.N)
        tmp=format(i,digit_num+'b')
        for j in range(modem.N):
            mat[i,j]=int(tmp[j])
    return mat

mat=create_mat()
    

In [8]:
zeros_key,ones_key=key_preparation(mysys.modem)
print(modem.code_book)
print(zeros_key)
print(ones_key)

{'00': (-0.7071067811865475+0.7071067811865475j), '01': (-0.7071067811865475-0.7071067811865475j), '11': (0.7071067811865475-0.7071067811865475j), '10': (0.7071067811865475+0.7071067811865475j)}
[array([0, 1]), array([0, 2])]
[array([2, 3]), array([1, 3])]


In [9]:
'''
def llr2prob(llr):
    log=True
    p0=math.exp(llr)/(1+math.exp(llr))
    p1=1/(1+math.exp(llr))
    if log==True:
        p0=math.log(p0)
        p1=math.log(p1)
    return np.array([p0,p1])
'''

'\ndef llr2prob(llr):\n    log=True\n    p0=math.exp(llr)/(1+math.exp(llr))\n    p1=1/(1+math.exp(llr))\n    if log==True:\n        p0=math.log(p0)\n        p1=math.log(p1)\n    return np.array([p0,p1])\n'

In [10]:
#test
tmp=np.arange(10).reshape(2,5)
print(tmp)
index=np.array([[1,1],[1,1]])
tmp[index]

[[0 1 2 3 4]
 [5 6 7 8 9]]


array([[[5, 6, 7, 8, 9],
        [5, 6, 7, 8, 9]],

       [[5, 6, 7, 8, 9],
        [5, 6, 7, 8, 9]]])

In [38]:
#prepere constants
zeros_key,ones_key=key_preparation(modem) #key number of ones and zeros
mat=create_mat() #binary matrics

Lc,[zeros,ones]=mysys.main_func(EsNodB) #Lcはデインターリーブされていない
#decode 
EST_cwd,EX_info=mysys.dc.LDPC_decode(Lc[mysys.BICM_deint]) #MAPデコーダで出てきた外部値を取得

Pre_info=EX_info[mysys.BICM_int]+Lc#順番の入れ替えをして、事前値にする


def soft_demapper(Pre_info,Lc,modem,zeros_key,ones_key,zeros,ones):
    '''
    inputs:

    
    ----
    return:
    updated LLR from demapper
    '''

    symbol_num=int(len(Lc)/modem.N) #シンボルの長さ

    #print(Pre_info[:30])
    #Pre_info=Pre_info.reshape([modem.N,symbol_num],order='F') #各シンボルで受信したビットごとに並べ替える　(symbol_num*bits_in_symbol)

    Pre_info_mat=mat@(Pre_info.reshape([modem.N,symbol_num],order='F'))#+Lc.reshape([modem.N,symbol_num],order='F'))

    ex_mat_z=np.zeros(zeros.shape) #the matrix of symbol generate probability of the bit zero
    ex_mat_o=np.zeros(zeros.shape) #bit ones

    for i in range(symbol_num): #1シンボル毎
        for j in range(modem.N): #シンボル内の1ビットごと
            ex_mat_z[:,i,j]-=Pre_info_mat[zeros_key[j],i]
            ex_mat_o[:,i,j]-=Pre_info_mat[ones_key[j],i]

    num=np.exp(-1*np.array(zeros)/No+ex_mat_z)
    num=np.sum(num,axis=0,keepdims=True)
    num=np.clip(num,10**(-15),10**15)
    num=np.log(num)

    den=np.exp(-1*np.array(ones)/No+ex_mat_o)
    den=np.sum(den,axis=0,keepdims=True)
    den=np.clip(den,10**(-15),10**15)
    den=np.log(den)

    res_Lc=(np.transpose(num[0]) - np.transpose(den[0])).ravel(order='F')  
    
    return res_Lc-Pre_info


K=512
M=4
EsNodB=0
EsNo = 10 ** (EsNodB / 10)
No=1/EsNo
mysys=Mysystem_LDPC(M,K)

print(soft_demapper(Pre_info,Lc,modem,zeros_key,ones_key,zeros,ones))
print(Lc)


ValueError: operands could not be broadcast together with shapes (512,) (2,) 

In [11]:
import numpy as np
a=np.log(2)
print(a)

0.6931471805599453


In [15]:
print("hello")
print(np.log(10**15))

hello
34.538776394910684


In [70]:
class BICM_ID:
    def __init__(self,modem,zeros,ones):
        self.modem=modem
        self.zeros=zeros
        self.ones=ones
        self.zeros_key,self.ones_key=self.key_preparation(self.modem)
        self.mat=self.create_mat(self.modem)
        print("mat shape")
        print(self.mat.shape)
    
    @staticmethod
    def key_preparation(modem):
        """ Creates the coordinates
        where either zeros or ones can be placed in the signal constellation..
        Returns
        -------
        zeros : list of lists of complex values
            The coordinates where zeros can be placed in the signal constellation.
        ones : list of lists of complex values
            The coordinates where ones can be placed in the signal constellation.
        """

        zeros = [[] for i in range(modem.N)]
        ones = [[] for i in range(modem.N)]

        bin_seq = modem.de2bin(modem.m)

        for bin_idx, bin_symb in enumerate(bin_seq):
            if modem.bin_input == True:
                key = bin_symb
            else:
                key = bin_idx
            for possition, digit in enumerate(bin_symb):
                if digit == '0':
                    zeros[possition].append(key)
                else:
                    ones[possition].append(key)
        
        #from str list to int array 
        for i in range(len(zeros)):
            zeros[i]=np.array([int(s, 2) for s in zeros[i]])
            ones[i]=np.array([int(s, 2) for s in ones[i]])
                    
        return zeros, ones
    
    @staticmethod
    def create_mat(modem):
        '''
        return 2D numpy array
        in which the first column is 0(bin)
        second column is1(bin),,,, 
        '''
        #print(str(0)+str(modem.N)+'b')
        #print(format(4,str(0)+str(modem.N)+'b'))
        mat=np.zeros((modem.M,modem.N),dtype=int)
        for i in range(modem.M):
            digit_num=str(0)+str(modem.N)
            tmp=format(i,digit_num+'b')
            for j in range(modem.N):
                mat[i,j]=int(tmp[j])
        return mat

    def demapper(self,Pre_info,Lc):
        '''
        inputs:

        
        ----
        return:
        updated LLR from demapper
        '''

        symbol_num=int(len(Lc)/self.modem.N) #シンボルの長さ

        #print(Pre_info[:30])
        #Pre_info=Pre_info.reshape([modem.N,symbol_num],order='F') #各シンボルで受信したビットごとに並べ替える　(symbol_num*bits_in_symbol)

        print(Pre_info.reshape([self.modem.N,symbol_num],order='F').shape)
        
        Pre_info_mat=self.mat@(Pre_info.reshape([self.modem.N,symbol_num],order='F'))#+Lc.reshape([modem.N,symbol_num],order='F'))
        

        print(self.modem.N)
        print(Pre_info_mat.shape)
        ex_mat_z=np.zeros(self.zeros.shape) #the matrix of symbol generate probability of the bit zero
        ex_mat_o=np.zeros(self.zeros.shape) #bit ones
        
        print(self.zeros.shape)

        for i in range(symbol_num): #1シンボル毎
            for j in range(self.modem.N): #シンボル内の1ビットごと
                ex_mat_z[:,i,j]-=Pre_info_mat[self.zeros_key[j],i]
                ex_mat_o[:,i,j]-=Pre_info_mat[self.ones_key[j],i]

        num=np.exp(-1*np.array(self.zeros)/No+ex_mat_z)
        num=np.sum(num,axis=0,keepdims=True)
        num=np.clip(num,10**(-15),10**15)
        num=np.log(num)

        den=np.exp(-1*np.array(self.ones)/No+ex_mat_o)
        den=np.sum(den,axis=0,keepdims=True)
        den=np.clip(den,10**(-15),10**15)
        den=np.log(den)

        res_Lc=(np.transpose(num[0]) - np.transpose(den[0])).ravel(order='F')  
        
        return res_Lc-Pre_info

K=512
M=4096
EsNodB=0
EsNo = 10 ** (EsNodB / 10)
No=1/EsNo
mysys=Mysystem_LDPC(M,K)
modem=mysys.modem
Lc,[zeros,ones]=mysys.main_func(EsNodB) #Lcはデインターリーブされていない
#decode 
EST_cwd,EX_info=mysys.dc.LDPC_decode(Lc[mysys.BICM_deint]) #MAPデコーダで出てきた外部値を取得

Pre_info=EX_info[mysys.BICM_int]+Lc#順番の入れ替えをして、事前値にする
print(Pre_info.shape)
print(Lc.shape)

dmp=BICM_ID(modem,zeros,ones)
print(dmp.demapper(Pre_info,Lc))
print(Lc)

(2, 4)
27
 itr 2

In [31]:
class BICM_ID:
    def __init__(self,modem,zeros,ones):
        self.modem=modem
        self.zeros=zeros
        self.ones=ones
        self.zeros_key,self.ones_key=self.key_preparation(modem)
    
    @staticmethod
    def key_preparation(modem):
        """ Creates the coordinates
        where either zeros or ones can be placed in the signal constellation..
        Returns
        -------
        zeros : list of lists of complex values
            The coordinates where zeros can be placed in the signal constellation.
        ones : list of lists of complex values
            The coordinates where ones can be placed in the signal constellation.
        """

        zeros = [[] for i in range(modem.N)]
        ones = [[] for i in range(modem.N)]

        bin_seq = modem.de2bin(modem.m)

        for bin_idx, bin_symb in enumerate(bin_seq):
            if modem.bin_input == True:
                key = bin_symb
            else:
                key = bin_idx
            for possition, digit in enumerate(bin_symb):
                if digit == '0':
                    zeros[possition].append(key)
                else:
                    ones[possition].append(key)
        
        #from str list to int array 
        for i in range(len(zeros)):
            zeros[i]=np.array([int(s, 2) for s in zeros[i]])
            ones[i]=np.array([int(s, 2) for s in ones[i]])
                    
        return zeros, ones

BICM_ID=BICM_ID(modem,zeros,ones)

In [251]:
Lc,[zeros,ones]=mysys.main_func(EsNodB) #Lcはデインターリーブされていない
#decode 
EST_cwd,EX_info=mysys.dc.LDPC_decode(Lc[mysys.BICM_deint]) #MAPデコーダで出てきた外部値を取得

Pre_info=EX_info[mysys.BICM_int]#順番の入れ替えをして、事前値にする

#lambda_Lc=EX_info+Lc #更新されたLLRの値

symbol_num=int(len(Lc)/modem.N) #シンボルの長さ
print(symbol_num)

Pre_info=Pre_info.reshape(symbol_num,modem.N) #各シンボルで受信したビットごとに並べ替える　(symbol_num*bits_in_symbol)

delta_St=np.zeros((M,symbol_num)) #各信号点の生起確率を計算する
for i in range(symbol_num): 
    delta_St[:,i]=-1*mat@Pre_info[i,:] #i番目のシンボルにおいて、それぞれのシンボルの正規確率の差分を求める

llr2prob(EX_info)

print(mat)
print(Pre_info[0,:])
print(delta_St[:,0])

512


TypeError: only size-1 arrays can be converted to Python scalars

In [233]:
print(delta_St)

[[  0.           0.           0.         ...   0.           0.
    0.        ]
 [ -9.24119333  -9.02917316   9.8219848  ...  -4.77631019  -3.08403718
    7.42137948]
 [-13.66339525 -12.24321817  21.69475357 ... -16.22463737   5.93417585
   -3.26593909]
 [ -4.42220192  -3.21404501  11.87276878 ... -11.44832718   9.01821303
  -10.68731858]]


In [234]:
a=modem.calc_exp(zeros,No)
b=modem.calc_exp(ones,No)
print(a.shape)
print(a.ravel().shape)
Lc3=(np.transpose(a[0]) - np.transpose(b[0])).ravel(order='F')
if np.any(Lc3!=Lc):
    print("error!")

(1, 512, 2)
(1024,)


In [224]:
LLR = []
for i in range(modem.N):#すべてのシンボルのiビット目のビットについて考える
    new_zeros=zeros[:,:,i]+delta_St[zeros_key[i],:]
    new_ones=ones[:,:,i]+delta_St[ones_key[i],:]
    
    num_post=modem.calc_exp(new_zeros,No)
    denum_post=modem.calc_exp(new_ones,No)
    llr = np.transpose(num_post[0]) - np.transpose(denum_post[0]) #二次元配列になってしまっているので、1次元に直す
    LLR.append(llr)

result = np.zeros((symbol_num * len(zeros_key)))
for i, llr_i in enumerate(LLR):
    result[i::len(zeros_key)] = llr_i

print(result.shape)

(1024,)


In [237]:


print(result-Lc-Pre_info.ravel())

[-12.24606445 -18.44567462  -5.03469357 ...  -3.68426571  -9.81412753
   5.50035485]
